# Demographic EDA (Universal 14-Step Framework)
This notebook is aligned to the same 14-step approach used in enrollment EDA. Run top-to-bottom.


## STEP 1 - Business Understanding


In [4]:
# Aim: Define business context for demographic table.
# Expected Output: Business context table.
# What You Get: Clear semantic definition of row meaning and usage.
# Data Engineer Learning: Business meaning should be explicit before technical checks.

import pandas as pd
import numpy as np
from pathlib import Path

business_context = pd.DataFrame([
    {'field': 'row_definition', 'value': 'One row represents demographic counts for one date-state-district-pincode record.'},
    {'field': 'table_type', 'value': 'Aggregated event summary table.'},
    {'field': 'measured_values', 'value': 'Demographic count columns (numeric measures).'},
    {'field': 'decisions_supported', 'value': 'Population distribution insights, quality monitoring, geographic comparisons.'},
])
display(business_context)


,field,value
0,row_definition,One row represents demographic counts for one ...
1,table_type,Aggregated event summary table.
2,measured_values,Demographic count columns (numeric measures).
3,decisions_supported,"Population distribution insights, quality moni..."


## STEP 2 - Structural Profiling


In [5]:
# Aim: Load demographic table with robust path handling.
# Expected Output: Base dataframe loaded and row/column count printed.
# What You Get: Stable data source initialization.
# Data Engineer Learning: Portable path logic reduces environment-specific breakage.

candidate_files = [
    Path('scripts/EDA/panda_eda/data/data_aadhar_demographic_full.csv'),
    Path('scripts/EDA/panda_eda/eda_enrollment/data/data_aadhar_demographic_full.csv'),
    Path('data/data_aadhar_demographic_full.csv'),
]

demographic_path = next((f for f in candidate_files if f.exists()), None)
if demographic_path is None:
    raise FileNotFoundError(f'Could not find demographic file in: {candidate_files}')

data_aadhar_demographic_full = pd.read_csv(demographic_path)
print('Demographic path:', demographic_path)
print('Rows, Cols:', data_aadhar_demographic_full.shape)


Demographic path: data\data_aadhar_demographic_full.csv
Rows, Cols: (2071700, 6)


In [6]:
# Aim: Run structural profiling checks.
# Expected Output: shape, columns, info, head, and sample rows.
# What You Get: Schema overview and first quality signals.
# Data Engineer Learning: Early schema visibility prevents downstream assumptions.

df = data_aadhar_demographic_full
print('Shape:', df.shape)
print('\nColumns:')
print(list(df.columns))
print('\nInfo:')
print(df.info())
print('\nHead:')
display(df.head())
print('\nSample:')
display(df.sample(min(5, len(df)), random_state=42))


Shape: (2071700, 6)

Columns:
['date', 'state', 'district', 'pincode', 'demo_age_5_17', 'demo_age_17_']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071700 entries, 0 to 2071699
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   date           object
 1   state          object
 2   district       object
 3   pincode        int64 
 4   demo_age_5_17  int64 
 5   demo_age_17_   int64 
dtypes: int64(3), object(3)
memory usage: 94.8+ MB
None

Head:


,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,01-03-2025,Uttar Pradesh,Gorakhpur,273213,49,529
1,01-03-2025,Andhra Pradesh,Chittoor,517132,22,375
2,01-03-2025,Gujarat,Rajkot,360006,65,765
3,01-03-2025,Andhra Pradesh,Srikakulam,532484,24,314
4,01-03-2025,Rajasthan,Udaipur,313801,45,785



Sample:


,date,state,district,pincode,demo_age_5_17,demo_age_17_
1728576,15-09-2025,Madhya Pradesh,Indore,453332,0,2
594264,01-11-2025,West Bengal,Haora,711415,0,22
1254164,02-09-2025,West Bengal,Uttar Dinajpur,733207,25,415
734291,13-12-2025,Telangana,Jayashankar Bhupalpally,506168,0,16
1368725,27-10-2025,Maharashtra,Palghar,401506,0,1


## STEP 3 - Grain Identification


In [7]:
# Aim: Identify natural key and test uniqueness.
# Expected Output: full duplicate count and key duplicate count.
# What You Get: Evidence for row grain.
# Data Engineer Learning: Grain clarity is mandatory for modeling correctness.

df = data_aadhar_demographic_full.copy()
natural_key = ['date', 'state', 'district', 'pincode']
full_dups = int(df.duplicated().sum())
key_dups = int(df.duplicated(subset=natural_key).sum())
print('Natural Key:', natural_key)
print('Full-row duplicates:', full_dups)
print('Key-level duplicates:', key_dups)


Natural Key: ['date', 'state', 'district', 'pincode']
Full-row duplicates: 473601
Key-level duplicates: 473601


## STEP 4 - Duplicate Analysis


In [8]:
# Aim: Inspect duplicate groups and build deduplicated working table.
# Expected Output: duplicate sample + before/after key duplicate counts.
# What You Get: Clean working table for downstream steps.
# Data Engineer Learning: Keep raw and deduplicated datasets separate for traceability.

df = data_aadhar_demographic_full.copy()
natural_key = ['date', 'state', 'district', 'pincode']

dup_view = df[df.duplicated(subset=natural_key, keep=False)].sort_values(by=['date', 'state'])
print('Duplicate group rows:', len(dup_view))
display(dup_view.head(20))

df_dedup = df.drop_duplicates(subset=natural_key, keep='first').reset_index(drop=True)
print('Key duplicates before:', int(df.duplicated(subset=natural_key).sum()))
print('Key duplicates after :', int(df_dedup.duplicated(subset=natural_key).sum()))


Duplicate group rows: 947202


,date,state,district,pincode,demo_age_5_17,demo_age_17_
418053,01-03-2025,Andaman and Nicobar Islands,North And Middle Andaman,744202,10,201
831438,01-03-2025,Andaman and Nicobar Islands,Nicobar,744301,16,180
1245525,01-03-2025,Andaman and Nicobar Islands,North And Middle Andaman,744202,10,201
1658910,01-03-2025,Andaman and Nicobar Islands,Nicobar,744301,16,180
414196,01-03-2025,Andhra Pradesh,Kurnool,518405,54,223
414208,01-03-2025,Andhra Pradesh,West Godavari,534111,10,133
414273,01-03-2025,Andhra Pradesh,Anantapur,515401,25,448
414285,01-03-2025,Andhra Pradesh,East Godavari,533251,11,133
414289,01-03-2025,Andhra Pradesh,Bapatla,523169,11,64
414291,01-03-2025,Andhra Pradesh,Guntur,522124,56,687


Key duplicates before: 473601
Key duplicates after : 0


## STEP 5 - Missing Value Analysis


In [9]:
# Aim: Analyze nulls, null%, and empty-string fields.
# Expected Output: missingness tables and state empty count.
# What You Get: Completeness risk baseline.
# Data Engineer Learning: Null and empty-string defects must both be tracked.

df = df_dedup.copy()
null_count = df.isnull().sum().sort_values(ascending=False)
null_pct = (df.isnull().sum() / max(len(df),1) * 100).sort_values(ascending=False)
missing_tbl = pd.DataFrame({'null_count': null_count, 'null_pct': null_pct})
display(missing_tbl.head(20))

if 'state' in df.columns:
    empty_state = int(df['state'].astype(str).str.strip().eq('').sum())
    print('Empty-string state rows:', empty_state)


,null_count,null_pct
date,0,0.0
state,0,0.0
district,0,0.0
pincode,0,0.0
demo_age_5_17,0,0.0
demo_age_17_,0,0.0


Empty-string state rows: 0


## STEP 6 - Data Type and Format Validation


In [10]:
# Aim: Validate date type, numeric columns, and pincode format.
# Expected Output: datetime parsing status, pincode length distribution, object columns.
# What You Get: Data contract validation evidence.
# Data Engineer Learning: Format rules should be explicit and measurable.

df = df_dedup.copy()
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)
print('Null dates after parse:', int(df['date'].isna().sum()))

pin_len = df['pincode'].astype(str).str.replace(r'\.0$','', regex=True).str.strip().str.len().value_counts(dropna=False).sort_index()
print('Pincode length distribution:')
display(pin_len.to_frame('count'))

obj_cols = df.select_dtypes(include=['object']).columns.tolist()
print('Object columns:', obj_cols)


Null dates after parse: 0
Pincode length distribution:


,count
pincode,
6,1598099


Object columns: ['state', 'district']


## STEP 7 - Domain Validation


In [11]:
# Aim: Validate value ranges and domain sanity.
# Expected Output: negative-measure diagnostics, date range, district sample.
# What You Get: Domain correctness signal.
# Data Engineer Learning: Domain checks separate data issues from genuine business events.

df = df_dedup.copy()
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)

measure_cols = [c for c in df.columns if c not in ['date','state','district','pincode'] and pd.api.types.is_numeric_dtype(df[c])]
if measure_cols:
    neg_rows = int((df[measure_cols] < 0).any(axis=1).sum())
else:
    neg_rows = 0

print('Numeric measure columns:', measure_cols)
print('Rows with negative values:', neg_rows)
print('Date range:', df['date'].min(), 'to', df['date'].max())
print('District sample:')
display(pd.Series(df['district'].dropna().astype(str).unique()).head(20).to_frame('district'))


Numeric measure columns: ['demo_age_5_17', 'demo_age_17_']
Rows with negative values: 0
Date range: 2025-03-01 00:00:00 to 2025-12-29 00:00:00
District sample:


,district
0,Gorakhpur
1,Chittoor
2,Rajkot
3,Srikakulam
4,Udaipur
5,Sikar
6,Tumakuru
7,Kurnool
8,Paschim Medinipur
9,Ghazipur


## STEP 8 - Cardinality and Distribution


In [12]:
# Aim: Compute key cardinality and distribution metrics.
# Expected Output: unique states, districts-per-state, unique pincodes.
# What You Get: Dimensional scaling profile.
# Data Engineer Learning: Cardinality affects partition strategy and indexing.

df = df_dedup.copy()
print('Unique states:', df['state'].nunique())
print('Unique pincodes:', df['pincode'].nunique())
state_district = df.groupby('state')['district'].nunique().sort_values(ascending=False)
display(state_district.head(20).to_frame('district_nunique'))


Unique states: 65
Unique pincodes: 19742


,district_nunique
state,
Uttar Pradesh,90
Madhya Pradesh,60
West Bengal,59
Karnataka,54
Maharashtra,53
Bihar,47
Rajasthan,46
Odisha,46
Andhra Pradesh,45


## STEP 9 - Cross-Column Consistency


In [13]:
# Aim: Test geo relationship stability.
# Expected Output: pincode->district and district->state uniqueness checks.
# What You Get: Referential consistency profile.
# Data Engineer Learning: Unstable relationships create join-quality issues in warehouse models.

df = df_dedup.copy()
pin_to_district = df.groupby('pincode')['district'].nunique().sort_values(ascending=False)
dist_to_state = df.groupby('district')['state'].nunique().sort_values(ascending=False)

print('Pincodes with >1 district:', int((pin_to_district > 1).sum()))
print('Districts with >1 state  :', int((dist_to_state > 1).sum()))

display(pin_to_district.head(20).to_frame('district_nunique'))
display(dist_to_state.head(20).to_frame('state_nunique'))


Pincodes with >1 district: 7708
Districts with >1 state  : 82


,district_nunique
pincode,
500037,7
500055,7
509339,7
500087,7
500090,7
509340,7
853204,7
721144,7
450661,6


,state_nunique
district,
Hooghly,6
South 24 Parganas,4
HOOGHLY,3
Diu,3
hooghly,3
Howrah,3
Daman,3
Kargil,3
Gajapati,3


## STEP 10 - Measures vs Dimensions Classification


In [15]:
# Aim: Classify columns into dimensions and measures.
# Expected Output: data dictionary table.
# What You Get: Fact/dimension readiness map.
# Data Engineer Learning: Explicit column roles reduce modeling ambiguity.

df = df_dedup.copy()
measure_cols = [c for c in df.columns if c not in ['date','state','district','pincode'] and pd.api.types.is_numeric_dtype(df[c])]
classification = []
for c in df.columns:
    if c == 'date':
        role = 'Time Dimension'
    elif c in ['state','district','pincode']:
        role = 'Location Dimension'
    elif c in measure_cols:
        role = 'Measure'
    else:
        role = 'Attribute'
    classification.append({'column': c, 'role': role, 'dtype': str(df[c].dtype)})

display(pd.DataFrame(classification))
print('dtypes:')
print(df.dtypes)
print('\nDescribe:')
display(df.describe(include='all'))



,column,role,dtype
0,date,Time Dimension,object
1,state,Location Dimension,object
2,district,Location Dimension,object
3,pincode,Location Dimension,int64
4,demo_age_5_17,Measure,int64
5,demo_age_17_,Measure,int64


dtypes:
date             object
state            object
district         object
pincode           int64
demo_age_5_17     int64
demo_age_17_      int64
dtype: object

Describe:


,date,state,district,pincode,demo_age_5_17,demo_age_17_
count,1598099,1598099,1598099,1.598099e+06,1.598099e+06,1.598099e+06
unique,95,65,983,NaN,NaN,NaN
top,12-12-2025,Andhra Pradesh,Pune,NaN,NaN,NaN
freq,23040,154753,9634,NaN,NaN,NaN
mean,NaN,NaN,NaN,5.266221e+05,2.251260e+00,2.064942e+01
std,NaN,NaN,NaN,1.974751e+05,1.391927e+01,1.177073e+02
min,NaN,NaN,NaN,1.000000e+05,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,3.961300e+05,0.000000e+00,2.000000e+00
50%,NaN,NaN,NaN,5.242230e+05,1.000000e+00,6.000000e+00
75%,NaN,NaN,NaN,6.915730e+05,2.000000e+00,1.500000e+01


## STEP 11 - Outlier Detection (Optional Advanced)


In [16]:
# Aim: Detect outliers using total_demographic and z-score.
# Expected Output: outlier count and top outlier rows.
# What You Get: anomaly candidates for investigation.
# Data Engineer Learning: Outliers can be quality defects or real-world shifts.

df = df_dedup.copy()
measure_cols = [c for c in df.columns if c not in ['date','state','district','pincode'] and pd.api.types.is_numeric_dtype(df[c])]
if measure_cols:
    df['total_demographic'] = df[measure_cols].fillna(0).sum(axis=1)
    mu = df['total_demographic'].mean()
    sd = df['total_demographic'].std()
    df['z'] = (df['total_demographic'] - mu) / (sd if pd.notna(sd) and sd != 0 else np.nan)
    outliers = df[df['z'].abs() > 3]
    print('Outlier rows (|z|>3):', len(outliers))
    display(outliers[['date','state','district','pincode','total_demographic','z']].head(20))
else:
    print('No numeric measure columns found for outlier detection.')


Outlier rows (|z|>3): 9384


,date,state,district,pincode,total_demographic,z
0,01-03-2025,Uttar Pradesh,Gorakhpur,273213,578,4.278272
2,01-03-2025,Gujarat,Rajkot,360006,830,6.220491
4,01-03-2025,Rajasthan,Udaipur,313801,830,6.220491
6,01-03-2025,Karnataka,Tumakuru,572201,420,3.060532
7,01-03-2025,Uttar Pradesh,Gorakhpur,273211,897,6.736875
8,01-03-2025,Andhra Pradesh,Kurnool,518313,1069,8.062517
10,01-03-2025,Uttar Pradesh,Ghazipur,233226,1725,13.118453
11,01-03-2025,Gujarat,Patan,385360,836,6.266735
13,01-03-2025,Rajasthan,Ganganagar,335027,570,4.216614
21,01-03-2025,Tamil Nadu,Kancheepuram,603202,742,5.542256


## STEP 12 - Trend and Time Analysis (Optional)


In [17]:
# Aim: Build monthly trend for total demographic counts.
# Expected Output: month-wise total trend table.
# What You Get: time-series movement view.
# Data Engineer Learning: Trend baselines are required for monitoring and anomaly alerts.

df = df_dedup.copy()
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)
measure_cols = [c for c in df.columns if c not in ['date','state','district','pincode'] and pd.api.types.is_numeric_dtype(df[c])]
if measure_cols:
    df['total_demographic'] = df[measure_cols].fillna(0).sum(axis=1)
    df['month'] = df['date'].dt.to_period('M').astype(str)
    monthly = df.groupby('month', as_index=False)['total_demographic'].sum().sort_values('month')
    display(monthly.head(24))
    print('Total months:', monthly['month'].nunique())
else:
    print('No numeric measure columns found for monthly trend.')


,month,total_demographic
0,2025-03,8190152
1,2025-04,907282
2,2025-05,939768
3,2025-06,1040944
4,2025-07,1510892
5,2025-09,5973259
6,2025-10,3833155
7,2025-11,7084305
8,2025-12,7117802


Total months: 9


## STEP 13 - Data Quality Risk Summary


In [18]:
# Aim: Build quality risk snapshot dictionary/table.
# Expected Output: compact summary object and table.
# What You Get: one-glance decision summary.
# Data Engineer Learning: End each EDA with a concise quality status snapshot.

df = df_dedup.copy()
summary = {
    'rows': len(df),
    'duplicates_key': int(df.duplicated(subset=['date','state','district','pincode']).sum()),
    'null_cells': int(df.isnull().sum().sum()),
    'unique_states': int(df['state'].nunique())
}
print(summary)

summary_tbl = pd.DataFrame([{'metric': k, 'value': v} for k, v in summary.items()])
display(summary_tbl)


{'rows': 1598099, 'duplicates_key': 0, 'null_cells': 0, 'unique_states': 65}


,metric,value
0,rows,1598099
1,duplicates_key,0
2,null_cells,0
3,unique_states,65


## STEP 14 - Documentation


In [19]:
# Aim: Create final EDA documentation artifact.
# Expected Output: findings table + markdown export.
# What You Get: durable documentation for handoff/interviews.
# Data Engineer Learning: EDA is complete only when findings are documented and saved.

df = df_dedup.copy()

gr_text = 'One row = demographic counts for one date-state-district-pincode combination.'
nk_text = '(date, state, district, pincode)'
issues = [
    f"Full-row duplicates: {int(df.duplicated().sum())}",
    f"Key duplicates: {int(df.duplicated(subset=['date','state','district','pincode']).sum())}",
    f"Total null cells: {int(df.isnull().sum().sum())}",
]
fixes = [
    'Enforce natural-key uniqueness in ingestion layer.',
    'Enforce date and pincode format contracts.',
    'Track pincode referential conflicts as data quality KPI.'
]
model_dir = [
    'Use date+location grain for conformed joins.',
    'Publish only quality-checked records to downstream models.'
]

eda_findings_doc = pd.DataFrame([
    {'section': 'Grain', 'details': gr_text},
    {'section': 'Natural Key', 'details': nk_text},
    {'section': 'Identified Issues', 'details': ' | '.join(issues)},
    {'section': 'Expected Fixes', 'details': ' | '.join(fixes)},
    {'section': 'Modeling Direction', 'details': ' | '.join(model_dir)},
])
display(eda_findings_doc)

report_dir = Path('scripts/EDA/panda_eda/consistency_reports')
report_dir.mkdir(parents=True, exist_ok=True)

csv_path = report_dir / 'final_demographic_eda_findings_table.csv'
md_path = report_dir / 'final_demographic_eda_findings.md'
eda_findings_doc.to_csv(csv_path, index=False)

md_lines = ['# Final Demographic EDA Findings', '']
for _, r in eda_findings_doc.iterrows():
    md_lines.append(f"## {r['section']}")
    md_lines.append(str(r['details']))
    md_lines.append('')
md_path.write_text('\n'.join(md_lines), encoding='utf-8')

print('Saved:')
print('-', csv_path)
print('-', md_path)


,section,details
0,Grain,One row = demographic counts for one date-stat...
1,Natural Key,"(date, state, district, pincode)"
2,Identified Issues,Full-row duplicates: 0 | Key duplicates: 0 | T...
3,Expected Fixes,Enforce natural-key uniqueness in ingestion la...
4,Modeling Direction,Use date+location grain for conformed joins. |...


Saved:
- scripts\EDA\panda_eda\consistency_reports\final_demographic_eda_findings_table.csv
- scripts\EDA\panda_eda\consistency_reports\final_demographic_eda_findings.md
